In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paradox-ml-challenge/sample_submission.csv
/kaggle/input/paradox-ml-challenge/train.csv
/kaggle/input/paradox-ml-challenge/test.csv


In [2]:
# This dataset is in HInglish which is kind of mix of both. I have very simplified 
# approch. I tried it doing with tf-idf, countvectorizer but accuracy and f1_score
# was 0.70 and 0.69 on validation set and test. And this is heavily dependent
# on how we preprocess the text. I made a different approach which is more 
# simplified and easy to implement i used embeddings but there is two ways one i 
# had to generate using BERT ot this way using fasttext. And Obviosly BERT has 
# more accuracy but this model got me .73 accuracy and f1_score and it is very 
# fast compared to BERT.


In [3]:
df=pd.read_csv("/kaggle/input/paradox-ml-challenge/train.csv")

In [4]:
df.head(12)

,Comment,Label
0,last proof me kaha gaya little confused about it,doubt
1,sir g ki value positive hogi,doubt
2,like the video guyssss,irrelevant
3,ummm sir i want see ur wife,irrelevant
4,physics wallah punabi mundamundi mein thusde d...,irrelevant
5,bhupesh,irrelevant
6,thx dedi,feedback
7,good mem thank you,feedback
8,sir hmko square ka center of mass wala nhi smj...,doubt
9,for constant pressure qcalorieducaloriewfor co...,feedback


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Label_encoded'] = le.fit_transform(df['Label'])
# Display the encoded labels doubt = 0
# feedback = 1
# irrelevant = 2
print(df)

                                                  Comment       Label  \
0       last proof me  kaha gaya little confused about it       doubt   
1                            sir g ki value positive hogi       doubt   
2                                  like the video guyssss  irrelevant   
3                             ummm sir i want see ur wife  irrelevant   
4       physics wallah punabi mundamundi mein thusde d...  irrelevant   
...                                                   ...         ...   
213742                             superb performance sir    feedback   
213743                                           mja aaya    feedback   
213744                           blocks wala sum be padao    feedback   
213745                    uminima stable umaxima unstable    feedback   
213746  sir you and namo sir get best physics teacher ...    feedback   

        Label_encoded  
0                   0  
1                   0  
2                   2  
3                   2  
4  

In [6]:
df_train = df[['Comment', 'Label_encoded','Label']]

In [7]:
pip install fasttext

Note: you may need to restart the kernel to use updated packages.


In [8]:
hinglish=df_train[['Comment']]
hinglish

,Comment
0,last proof me kaha gaya little confused about it
1,sir g ki value positive hogi
2,like the video guyssss
3,ummm sir i want see ur wife
4,physics wallah punabi mundamundi mein thusde d...
...,...
213742,superb performance sir
213743,mja aaya
213744,blocks wala sum be padao
213745,uminima stable umaxima unstable


In [9]:
other_dataframe=pd.read_csv("/kaggle/input/paradox-ml-challenge/test.csv")
other_comments = other_dataframe['Comment'].tolist()

comments = hinglish['Comment'].tolist()

all_comments = comments + other_comments

with open('comments.txt', 'w') as f:
    for comment in all_comments:
        f.write(comment + '\n')



In [10]:
import fasttext
model = fasttext.train_unsupervised('/kaggle/working/comments.txt', model='skipgram', dim=300,lr=0.1)
model.save_model('hinglish_vectors.bin')




Read 3M words
Number of words:  17059
Number of labels: 0
Progress: 100.0% words/sec/thread:   25549 lr:  0.000000 avg.loss:  2.030565 ETA:   0h 0m 0s


In [11]:
model = fasttext.load_model('hinglish_vectors.bin')

In [12]:
from numpy import dot
from numpy.linalg import norm
vector_1 = model.get_word_vector("acha")
vector_2 = model.get_word_vector("theek hai")

# Calculate cosine similarity
cosine_similarity = dot(vector_1, vector_2) / (norm(vector_1) * norm(vector_2))
print("Cosine Similarity between 'acha' and 'theek hai':", cosine_similarity)

Cosine Similarity between 'acha' and 'theek hai': 0.3663366


In [13]:
# this will get more better if we train it on more social media chats data.

In [14]:
# embedding vector
vector = model.get_word_vector("acha")
print(vector)

[ 2.83864960e-02 -1.76672235e-01 -3.16103280e-01  1.91389561e-01
  2.22050073e-03  1.13228150e-01  1.09812291e-02  5.10115743e-01
 -4.53551523e-02  7.20706582e-02 -3.34939640e-03  2.45126322e-01
 -9.99861434e-02 -1.51488855e-01 -4.03385051e-02 -1.28546894e-01
  1.65281415e-01  1.48856184e-02  7.41612315e-02  5.12613840e-02
 -2.82842785e-01 -1.11778393e-01  6.69482630e-03 -1.81044191e-01
 -1.21791877e-01 -2.53670067e-01 -4.34062164e-03 -2.10402727e-01
  5.19620022e-03  1.79992661e-01  1.11760437e-01  1.21156119e-01
 -3.16955477e-01 -1.49755478e-01 -1.92950651e-01  9.17118415e-02
 -5.07005863e-02  9.76370797e-02  2.08969396e-02  7.08697960e-02
  2.72915602e-01  1.83152229e-01  9.45280269e-02 -1.88271746e-01
  2.45877072e-01  2.27623922e-03  1.13077611e-01  1.42222539e-01
 -2.17095539e-01  5.54206848e-01 -2.62521118e-01  6.83012009e-02
 -4.19822745e-02 -9.68082622e-02  1.67763084e-01  1.48728983e-02
 -2.57093519e-01 -1.75103188e-01  1.00894302e-01 -3.52440774e-02
  2.06129193e-01 -1.63563

In [15]:
import pandas as pd
data = pd.read_csv('/kaggle/input/paradox-ml-challenge/train.csv')  
texts = data['Comment']  


In [16]:
def text_to_vector(text):
    words = text.split()  
    vectors = [model.get_word_vector(word) for word in words]  
    return sum(vectors) / len(vectors) if vectors else None  

data['vector'] = texts.apply(text_to_vector)


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

label_encoder = LabelEncoder()
data['encoded_label'] = label_encoder.fit_transform(data['Label'])  

X_train, X_test, y_train, y_test = train_test_split(data['vector'].tolist(), data['encoded_label'], test_size=0.2, random_state=42)


In [18]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, accuracy_score

clf = XGBClassifier(objective='multi:softmax', num_class=3, n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)

clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

print(f'F1 Score: {f1:.2f}')
print(f'Accuracy: {accuracy:.2f}')


F1 Score: 0.70
Accuracy: 0.70


In [19]:
df_test=pd.read_csv("/kaggle/input/paradox-ml-challenge/test.csv")
test=df_test['Comment']
def text_to_vector(text):
    words = text.split()  
    vectors = [model.get_word_vector(word) for word in words]  
    return sum(vectors) / len(vectors) if vectors else None  


df_test['vector'] = test.apply(text_to_vector)
okbabes=df_test['vector'].tolist()
predictions = clf.predict(okbabes) 


results_df = pd.DataFrame({
    'ID': df_test['ID'],
    'Comment': df_test['Comment'],
    'Predicted_Label': predictions
})

print(results_df)


          ID                                            Comment  \
0          0                            bhai  ko kiska paper ha   
1          1                                     thank me later   
2          2                      thankqqqqq mam for this video   
3          3  apki teaching se physics sahi main ek visuliza...   
4          4  i didnt understand the differentiation part co...   
...      ...                                                ...   
91602  91602  sir app great ho hamari help karne ke liye app...   
91603  91603                 sir there any english batch for us   
91604  91604  mam mene aapke channal ke video first time dek...   
91605  91605  literally amazing session i m prepare for nda ...   
91606  91606  machli pani me gyi chapak machali pani me gyi ...   

       Predicted_Label  
0                    2  
1                    2  
2                    1  
3                    1  
4                    0  
...                ...  
91602               

In [20]:
label_mapping = {
    0: 'doubt',
    1: 'feedback',
    2: 'irrelevant'
}

predicted_labels = [label_mapping[label] for label in predictions]


results_df = pd.DataFrame({
    'ID': df_test['ID'],
    'Comment': df_test['Comment'],
    'Label': predicted_labels
})

results_df

,ID,Comment,Label
0,0,bhai ko kiska paper ha,irrelevant
1,1,thank me later,irrelevant
2,2,thankqqqqq mam for this video,feedback
3,3,apki teaching se physics sahi main ek visuliza...,feedback
4,4,i didnt understand the differentiation part co...,doubt
...,...,...,...
91602,91602,sir app great ho hamari help karne ke liye app...,feedback
91603,91603,sir there any english batch for us,doubt
91604,91604,mam mene aapke channal ke video first time dek...,feedback
91605,91605,literally amazing session i m prepare for nda ...,feedback


In [21]:
df_final = results_df[['ID', 'Label']]
print(df_final)


          ID       Label
0          0  irrelevant
1          1  irrelevant
2          2    feedback
3          3    feedback
4          4       doubt
...      ...         ...
91602  91602    feedback
91603  91603       doubt
91604  91604    feedback
91605  91605    feedback
91606  91606  irrelevant

[91607 rows x 2 columns]


In [22]:
df_final.to_csv('submission.csv',index=False)

In [23]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

       doubt       0.71      0.71      0.71     11067
    feedback       0.70      0.75      0.73     19131
  irrelevant       0.70      0.62      0.66     12552

    accuracy                           0.70     42750
   macro avg       0.70      0.69      0.70     42750
weighted avg       0.70      0.70      0.70     42750

